## Homework #10

##### Author: [Radoslav Neychev](https://www.linkedin.com/in/radoslav-neychev/), @neychev

In [1]:
import numpy as np

import torch
from torch import nn
from torch.nn import functional as F

import torchvision
from torchvision.datasets import FashionMNIST

from matplotlib import pyplot as plt
from IPython.display import clear_output

### Task #1:
Let's return to the handwritten digit recognition problem discussed in the first lesson. We will also work with the [FashionMNIST](https://github.com/zalandoresearch/fashion-mnist) dataset. In this task we will use the entire dataset.

__Your main task: implement the entire model training pipeline and achieve a quality of $\geq 88.5\%$ on the test sample.__

There is no code for training the model in this task. There are only a few tests that will help you debug your solution. For an example, you can refer to the notebooks from previous classes.

In [ ]:
# do not change the code in the block below
# __________start of block__________
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# __________end of block__________

In [ ]:
# do not change the code in the block below
# __________start of block__________

train_fmnist_data = FashionMNIST('.', train=True, transform=torchvision.transforms.ToTensor(), download=True)
test_fmnist_data = FashionMNIST('.', train=False, transform=torchvision.transforms.ToTensor(), download=True)


train_data_loader = torch.utils.data.DataLoader(
    train_fmnist_data,
    batch_size=32,
    shuffle=True,
    num_workers=2
)

test_data_loader = torch.utils.data.DataLoader(
    test_fmnist_data,
    batch_size=32,
    shuffle=False,
    num_workers=2
)

random_batch = next(iter(train_data_loader))
_image, _label = random_batch[0][0], random_batch[1][0]
plt.figure()
plt.imshow(_image.reshape(28, 28))
plt.title(f'Image label: {_label}')
# __________end of block__________

Build the model below. Please do not build an over-complicated network; you should not make it deeper than four layers (fewer is possible). Your main task is to train the model and obtain quality on the deferred (test sample) of at least 88.5% accuracy.

__Attention, your model must be represented by the `model` variable. It should receive a tensor of dimension (1, 28, 28) as input.__

In [ ]:
# Creating model instance

# your code here

Don't forget to transfer the model to the selected `device`!

In [ ]:
model.to(device)

Local tests to validate your model are available below:

In [ ]:
# do not change the code in the block below
# __________start of block__________
assert model is not None, 'Please, use `model` variable to store your model'

try:
    x = random_batch[0].to(device)
    y = random_batch[1].to(device)

    # compute outputs given inputs, both are variables
    y_predicted = model(x)    
except Exception as e:
    print('Something is wrong with the model')
    raise e
    
    
assert y_predicted.shape[-1] == 10, 'Model should predict 10 logits/probas'

print('Everything seems fine!')
# __________end of block__________

Tune the model parameters on the training set. We also recommend working with `learning rate`.

In [ ]:
# your code here

Also, we remind you that you can refer to the excellent [documentation](https://pytorch.org/docs/stable/index.html) and [training examples](https://pytorch.org/tutorials/) at any time.

Let's evaluate the quality of classification:

In [ ]:
predicted_labels = []
real_labels = []
model.eval()
with torch.no_grad():
    for batch in train_data_loader:
        y_predicted = model(batch[0].to(device))
        predicted_labels.append(y_predicted.argmax(dim=1).cpu())
        real_labels.append(batch[1])

predicted_labels = torch.cat(predicted_labels)
real_labels = torch.cat(real_labels)
train_acc = (predicted_labels == real_labels).type(torch.FloatTensor).mean()

In [ ]:
print(f'Neural network accuracy on train set: {train_acc:3.5}')

In [ ]:
predicted_labels = []
real_labels = []
model.eval()
with torch.no_grad():
    for batch in test_data_loader:
        y_predicted = model(batch[0].to(device))
        predicted_labels.append(y_predicted.argmax(dim=1).cpu())
        real_labels.append(batch[1])

predicted_labels = torch.cat(predicted_labels)
real_labels = torch.cat(real_labels)
test_acc = (predicted_labels == real_labels).type(torch.FloatTensor).mean()

In [ ]:
print(f'Neural network accuracy on test set: {test_acc:3.5}')

Checking that the required thresholds have been passed:

In [ ]:
assert test_acc >= 0.885, 'Train accuracy is below 0.885 threshold'
assert train_acc >= 0.905, 'Test accuracy is below 0.905 while test accuracy is fine. We recommend to check your model and data flow'

### Submitting the task
Download the file `hw10_data_dict.npy` (the link is on the task page) and run the code below to generate the parcel. The code below can load it (but if there is an error, please download and load it manually).

In [ ]:
!wget https://github.com/girafe-ai/ml-course/raw/23s_dd_ml/homeworks/hw10_fmnist/hw10_data_dict.npy

In [ ]:
# do not change the code in the block below
# __________start of block__________
import os

assert os.path.exists('hw10_data_dict.npy'), 'Please, download `hw10_data_dict.npy` and place it in the working directory'

def get_predictions(model, eval_data, step=10):
    
    predicted_labels = []
    model.eval()
    with torch.no_grad():
        for idx in range(0, len(eval_data), step):
            y_predicted = model(eval_data[idx:idx+step].to(device))
            predicted_labels.append(y_predicted.argmax(dim=1).cpu())
    
    predicted_labels = torch.cat(predicted_labels)
    return predicted_labels

loaded_data_dict = np.load('hw10_data_dict.npy', allow_pickle=True)

submission_dict = {
    'train': get_predictions(model, torch.FloatTensor(loaded_data_dict.item()['train'])).numpy(),
    'test': get_predictions(model, torch.FloatTensor(loaded_data_dict.item()['test'])).numpy()
}

np.save('submission_dict_hw10.npy', submission_dict, allow_pickle=True)
print('File saved to `submission_dict_hw10.npy`')
# __________end of block__________

This completes the task. Congratulations!